In [1]:
import sys
sys.path.append("../src/")
from model import AEDist
import numpy as np
import torch
import phate
from heatgeo.embedding import HeatGeo
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from scipy.spatial import procrustes
import scanpy as sc
import scprep
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
import hydra
import os
from omegaconf import DictConfig, OmegaConf
import wandb
from hydra import initialize, compose
import pickle
from scipy.spatial.distance import pdist, squareform, cdist
import graphtools

In [2]:
data = np.load('../data/eb.npz')

In [3]:
# adata = sc.read('../data/old/eb.h5ad', backed='r')
adata = sc.read('../data/old/eb_hv.h5ad')

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
pca_loadings = adata.obsm['X_pca']

In [5]:
pca_components = adata.varm['PCs']

In [6]:
original_feature_means = np.mean(adata[:,:].X.toarray(), axis=0)

In [7]:
X_approx = pca_loadings @ pca_components.T + original_feature_means

In [8]:
np.savez('../data/eb_pcs.npz', pca_loadings=pca_loadings, pca_components=pca_components)

In [9]:
1 - ((adata[:,:].X.toarray() - X_approx)**2).mean()/adata[:,:].X.toarray().var()

0.38969232424440226

In [10]:
# Sanity check!
# Generate a synthetic data2set
np.random.seed(42)  # For reproducibility
data2 = np.random.rand(100, 10)  # 100 cells, 10 genes
adata2 = sc.AnnData(data2)
sc.tl.pca(adata2, svd_solver='full', n_comps=10)
pca_components2 = adata2.varm['PCs']
pca_loadings2 = adata2.obsm['X_pca']
original_mean2 = np.mean(adata2.X, axis=0)
reconstructed_data2 = pca_loadings2 @ pca_components2.T + original_mean2
((reconstructed_data2 - adata2.X)**2).mean()

5.034935516207318e-17